In [ ]:
# %%
import sys
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from wespeaker.cli.speaker import load_model

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.append(str(PROJECT_ROOT))

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("PROJECT_ROOT =", PROJECT_ROOT)
print("Using device:", DEVICE)

# Load WeSpeaker bundle + move backbone to GPU
speaker = load_model(PROJECT_ROOT / "wespeaker-voxceleb-resnet293-LM")
speaker.model = speaker.model.to(DEVICE).eval()

print("ResNet-293 loaded")

# WeSpeaker mel params
SAMPLE_RATE = 16000
N_MELS = 80
FRAME_LENGTH_MS = 25
FRAME_SHIFT_MS = 10
N_FFT = int(SAMPLE_RATE * FRAME_LENGTH_MS / 1000)
HOP_LENGTH = int(SAMPLE_RATE * FRAME_SHIFT_MS / 1000)

print(f"Mel params: n_fft={N_FFT}, hop_length={HOP_LENGTH}, n_mels={N_MELS}")

In [ ]:
# %%
# ========== LOAD YOUR TRAINED FC HEAD ==========
SPEAKERS = ["eden", "idan", "yoav"]
speaker_to_id = {s: i for i, s in enumerate(SPEAKERS)}
id_to_speaker = {i: s for s, i in speaker_to_id.items()}


class SpeakerHead(nn.Module):
    def __init__(self, in_dim=256, num_classes=3):
        super().__init__()
        self.fc = nn.Linear(in_dim, num_classes)

    def forward(self, x):
        return self.fc(x)


head = SpeakerHead(in_dim=256, num_classes=len(SPEAKERS)).to(DEVICE)

head_path = PROJECT_ROOT / "data" / "heads" / "resnet_293_speaker_head.pt"
assert head_path.exists(), f"Missing head checkpoint: {head_path}"

ckpt = torch.load(head_path, map_location="cpu")
head.load_state_dict(ckpt["state_dict"])
head = head.to(DEVICE).eval()

if "speakers" in ckpt:
    SPEAKERS = ckpt["speakers"]
    speaker_to_id = {s: i for i, s in enumerate(SPEAKERS)}
    id_to_speaker = {i: s for s, i in speaker_to_id.items()}

print("Loaded head from:", head_path)
print("Speakers:", SPEAKERS)

In [ ]:
class WeSpeakerWithHeadForGradCAM(nn.Module):
    def __init__(
        self, backbone: nn.Module, head: nn.Module, try_transpose: bool = True
    ):
        super().__init__()
        self.backbone = backbone
        self.head = head
        self.try_transpose = try_transpose

    def forward(self, feats):
        # Accept NCHW from Grad-CAM and squeeze channel dim
        if feats.ndim == 4 and feats.size(1) == 1:
            feats = feats.squeeze(1)

        # Run backbone
        try:
            out = self.backbone(feats)
        except Exception:
            if not self.try_transpose:
                raise
            out = self.backbone(feats.transpose(1, 2))

        # Unwrap tuple/list (WeSpeaker behavior)
        if isinstance(out, (tuple, list)):
            emb = out[-1]
        else:
            emb = out

        # ---- HARD SHAPE GUARD ----
        if emb.ndim == 0:
            emb = emb.unsqueeze(0).unsqueeze(0)
        elif emb.ndim == 1:
            emb = emb.unsqueeze(0)
        elif emb.ndim > 2:
            emb = emb.view(emb.size(0), -1)

        # ---- HARD DEVICE GUARD (THIS FIXES YOUR ERROR) ----
        emb = emb.to(next(self.head.parameters()).device)

        logits = self.head(emb)

        if logits.ndim == 1:
            logits = logits.unsqueeze(0)

        return logits


wrapped_model = WeSpeakerWithHeadForGradCAM(speaker.model, head).to(DEVICE).eval()

# Enable gradients for backbone (needed for Grad-CAM)
for p in wrapped_model.backbone.parameters():
    p.requires_grad_(True)

# Head grads not required (optional)
for p in wrapped_model.head.parameters():
    p.requires_grad_(False)

print("wrapped_model ready")

In [ ]:
# %%
# ========== HELPERS (your existing ones, unchanged) ==========
def upsample_hw(arr: np.ndarray, size_hw: tuple[int, int], mode: str = "bilinear") -> np.ndarray:
    arr = np.ascontiguousarray(arr)

    if arr.ndim == 2:
        t = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0).float()  # [1,1,H,W]
        t = F.interpolate(t, size=size_hw, mode=mode, align_corners=False)
        return t[0, 0].cpu().numpy()

    if arr.ndim == 3 and arr.shape[2] == 3:
        t = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0).float()  # [1,3,H,W]
        t = F.interpolate(t, size=size_hw, mode=mode, align_corners=False)
        return t[0].permute(1, 2, 0).cpu().numpy()

    raise ValueError(f"Unsupported shape: {arr.shape}")

def to_float01(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    if img.max() > 1.0:
        img = img / 255.0
    return np.clip(img, 0.0, 1.0)

def plot_mel(mel, save_path=None, title=None):
    if isinstance(mel, torch.Tensor):
        mel = mel.squeeze(0).detach().cpu().numpy()

    mel_db = 10.0 * np.log10(np.maximum(mel, 1e-10))
    mel_db = np.clip(mel_db, mel_db.max() - 80.0, mel_db.max())
    mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-8)

    plt.figure(figsize=(10, 4))
    plt.imshow(mel_norm, aspect="auto", cmap="magma", interpolation="bilinear", origin="lower")
    plt.colorbar(label="Normalized Energy")
    plt.xlabel("Time Frames")
    plt.ylabel("Mel Frequency Bins")
    if title:
        plt.title(title)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=200)
        plt.close()
    else:
        plt.show()

def load_wav_mono_16k(wav_path: str) -> torch.Tensor:
    wav, sr = torchaudio.load(wav_path)

    if wav.shape[0] > 1:
        wav = wav[:1, :]

    if sr != SAMPLE_RATE:
        wav = torchaudio.functional.resample(wav, sr, SAMPLE_RATE)

    return wav  # keep on CPU for mel transform stability


In [ ]:
# %%
# ========== FEATURE EXTRACTOR (mel) ==========
# Important: mel_transform must be on DEVICE if you want mel tensor on GPU.
mel_transform = T.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_mels=N_MELS,
    n_fft=N_FFT,
    hop_length=HOP_LENGTH,
).to(DEVICE)

In [ ]:
# %%
# ========== TARGET LAYERS (same idea, but via wrapped_model.backbone) ==========
target_layers = [
    # wrapped_model.backbone.layer1[9].conv3,
    # wrapped_model.backbone.layer2[19].conv3,
    # wrapped_model.backbone.layer3[63].conv3,
    wrapped_model.backbone.layer4[2].conv3,
]

layer_names = [
    # "layer1.9.conv3",
    # "layer2.19.conv3",
    # "layer3.63.conv3",
    "layer4.2.conv3",
]

print("Target layers:")
for n, l in zip(layer_names, target_layers):
    print(n, "->", l)

In [ ]:
# %%
def run_gradcam_speaker(
    wav_path: str,
    target_speaker: str,
    save_dir: str = "gradcam_results_cls",
    upscale: int = 10,
    cam_quantile: float = 0.85,
):
    assert target_speaker in speaker_to_id, f"Unknown speaker: {target_speaker}"

    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    fname = Path(wav_path).stem

    # 1) Load wav (CPU), compute mel ON GPU with grad
    wav_cpu = load_wav_mono_16k(wav_path)              # [1, T] CPU
    wav_gpu = wav_cpu.to(DEVICE)                       # move to GPU for mel_transform
    feats = mel_transform(wav_gpu)                     # [1, 80, Tfeat] on GPU
    feats = feats.contiguous().float()
    feats_cam = feats.unsqueeze(1)                  # [1, 1, 80, Tfeat] for Grad-CAM

    # 1b) Detached mel for visualization (numpy)
    with torch.no_grad():
        mel2d = feats.detach()[0].cpu().numpy()        # [80, Tfeat]
    mel_norm = (mel2d - mel2d.min()) / (mel2d.max() - mel2d.min() + 1e-8)

    originals_dir = save_dir / "original"
    originals_dir.mkdir(exist_ok=True)
    plot_mel(
        mel2d,
        save_path=originals_dir / f"{fname}_mel.png",
        title=f"{fname} (target={target_speaker})",
    )

    # 2) target class
    target = ClassifierOutputTarget(speaker_to_id[target_speaker])

    layer_stats = {}

    for layer, lname in zip(target_layers, layer_names):
        print(f"[Grad-CAM] Layer: {lname}")

        layer_dir = save_dir / lname
        layer_dir.mkdir(parents=True, exist_ok=True)

        wrapped_model.zero_grad(set_to_none=True)
        if feats.grad is not None:
            feats.grad.zero_()

        # Optional sanity check
        with torch.no_grad():
            tmp = wrapped_model(feats)
        print("wrapped_model(feats) shape:", tmp.shape)

        try:
            cam = GradCAM(model=wrapped_model, target_layers=[layer], use_cuda=(DEVICE == "cuda"))
        except TypeError:
            cam = GradCAM(model=wrapped_model, target_layers=[layer])

        with torch.enable_grad():
            grayscale_cam = cam(
                input_tensor=feats_cam,      # [1,1,80,T] for Grad-CAM
                targets=[target],
                aug_smooth=False,
                eigen_smooth=True,
            )[0]  # [H', W'] (feature map space)

        # 4) align CAM to mel for plotting
        cam_on_mel = upsample_hw(grayscale_cam, mel2d.shape, mode="bicubic")
        cam_norm = (cam_on_mel - cam_on_mel.min()) / (cam_on_mel.max() - cam_on_mel.min() + 1e-8)

        # 5) weighted mel
        mel_weighted = mel2d * cam_norm
        plot_mel(
            mel_weighted,
            save_path=layer_dir / f"{fname}_mel_weighted.png",
            title=f"{fname} | {lname} | weighted",
        )

        # 6) focus mask
        thr = np.quantile(cam_norm, cam_quantile)
        mask = cam_norm >= thr
        mel_focus = np.where(mask, mel2d, mel2d.min())
        plot_mel(
            mel_focus,
            save_path=layer_dir / f"{fname}_mel_focus_q{cam_quantile}.png",
            title=f"{fname} | {lname} | focus",
        )

        # 7) overlay
        H, W = mel_norm.shape
        H2, W2 = H * upscale, W * upscale

        rgb_big = upsample_hw(np.stack([mel_norm] * 3, axis=-1), (H2, W2), mode="bicubic")
        cam_big = upsample_hw(cam_norm, (H2, W2), mode="bicubic")

        rgb_big = to_float01(rgb_big)
        cam_big = to_float01(cam_big)
        overlay = show_cam_on_image(rgb_big, cam_big, use_rgb=True)
        plt.imsave(layer_dir / f"{fname}_overlay.png", overlay)

        # 8) stats
        entropy = -np.sum(cam_norm * np.log(cam_norm + 1e-8)) / cam_norm.size
        support = (cam_norm > 0.5).mean()
        layer_stats[lname] = {"entropy": float(entropy), "support@0.5": float(support)}

    stats_df = pd.DataFrame(layer_stats).T
    stats_df.to_csv(save_dir / "layer_cam_stats.csv")

    print("Saved Grad-CAM results to:", save_dir)
    print(stats_df)


In [ ]:
wav_dir = PROJECT_ROOT / "data" / "wavs"

# Use an existing audio file
test_wav = wav_dir / "idan_012.wav"
if not test_wav.exists():
    # Fallback to first available idan file
    idan_files = list(wav_dir.glob("idan_*.wav"))
    if idan_files:
        test_wav = sorted(idan_files)[0]
    else:
        raise FileNotFoundError(f"No audio files found in {wav_dir}")

print(f"Testing with: {test_wav}")

run_out_dir = Path(f"gradcam_results/idan/{test_wav.stem}")
originals_dir = run_out_dir / "original"

run_gradcam_speaker(
    wav_path=str(test_wav),
    target_speaker="idan",
    save_dir=str(run_out_dir),
    upscale=12,
)

# Uncomment to run on multiple speakers
# top3_by_speaker = {
#     "eden": ["eden_007.wav", "eden_010.wav", "eden_012.wav"],
#     "idan": ["idan_001.wav", "idan_002.wav", "idan_005.wav"],
#     "yoav": ["yoav_004.wav", "yoav_014.wav", "yoav_015.wav"],
# }

# for spk, fnames in top3_by_speaker.items():
#     for fname in fnames:
#         wav_path = wav_dir / fname
#         if not wav_path.exists():
#             print(f"[WARN] missing file: {wav_path}")
#             continue

#         run_gradcam_speaker(
#             wav_path=str(wav_path),
#             target_speaker=spk,
#             save_dir=f"gradcam_results/{spk}/{fname.replace('.wav', '')}",
#             upscale=12,
#         )